# SeaSenseLib Demo Notebook

This notebook demonstrates the key capabilities of SeaSenseLib for reading, processing, and visualizing oceanographic sensor data using the **modern API**.

## Overview

SeaSenseLib is a Python library for reading, converting, and plotting oceanographic sensor data from various instrument formats. It provides:

- **Modern API** with simple `ssl.read()` and `ssl.write()` functions
- **Readers** for multiple formats (CNV, NetCDF, RBR, ADCP, etc.)
- **Writers** for data export (NetCDF, CSV, Excel)  
- **Plotters** for oceanographic visualizations via `ssl.plot.*`
- **Processors** for data analysis and manipulation

## New API Features

The modern SeaSenseLib API simplifies common tasks:

```python
import seasenselib as ssl

# Read any supported format
data = ssl.read('sensor_data.cnv')

# Create oceanographic plots
ssl.plot('depth-profile', profile_data, title="CTD Profile")
ssl.plot('ts-diagram', profile_data, title="T-S Diagram")
ssl.plot('time-series', profile_data, parameters=['temperature'], title="CTD Profile T-S Diagram")

# Export to various formats
ssl.write(data, 'output.nc')   # NetCDF
ssl.write(data, 'output.csv')  # CSV
```


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

# Import SeaSenseLib using the new modern API
import seasenselib as ssl

# For processors, we still import directly since they don't have API shortcuts yet
from seasenselib.processors import SubsetProcessor, StatisticsProcessor

# Set up matplotlib for inline plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (10, 6)

print("✓ SeaSenseLib demo environment ready!")
print(f"✓ SeaSenseLib version: {ssl.__version__}")

## 1. Reading Data from Different Formats

SeaSenseLib can read data from various oceanographic instruments. Let's start with the example data files.

### 1.1 Reading CTD Profile Data (CNV Format)

First, let's read a vertical CTD profile from a SeaBird CNV file:

In [ ]:
# Read CTD profile data using the new API
profile_data = ssl.read("../examples/sea-practical-2023.cnv")

print("CTD Profile Dataset:")
print(f"Dimensions: {dict(profile_data.dims)}")
print(f"Data variables: {list(profile_data.data_vars)}")
print(f"Coordinates: {list(profile_data.coords)}")
print(f"\nData shape: {profile_data.sizes}")

### 1.2 Reading Time Series Data (Moored Instrument)

Now let's read time series data from a moored instrument:

In [ ]:
# Read time series data from moored instrument using the new API
timeseries_data = ssl.read("../examples/denmark-strait-ds-m1-17.cnv")

print("Time Series Dataset:")
print(f"Dimensions: {dict(timeseries_data.dims)}")
print(f"Data variables: {list(timeseries_data.data_vars)}")
print(f"Time range: {timeseries_data.time.min().values} to {timeseries_data.time.max().values}")
print(f"Duration: {len(timeseries_data.time)} measurements")

## 2. Data Exploration and Quality Assessment

Let's explore the data structure and assess data quality:

In [ ]:
# Display detailed information about the profile dataset
print("=== CTD Profile Data Details ===")
print(profile_data)

print("\n=== Available Parameters and Units ===")
for var in profile_data.data_vars:
    attrs = profile_data[var].attrs
    units = attrs.get('units', 'no units')
    long_name = attrs.get('long_name', var)
    print(f"{var}: {long_name} ({units})")

### 2.1 Basic Statistics

In [ ]:
# Calculate basic statistics for profile data
available_params = [var for var in profile_data.data_vars if var in ['temperature', 'salinity', 'pressure', 'oxygen']]

print("=== CTD Profile Statistics ===")
for param in available_params:
    try:
        stats_processor = StatisticsProcessor(profile_data, param)
        stats = stats_processor.get_all_statistics()
        
        print(f"\n{param.capitalize()}:")
        print(f"  Mean: {stats['mean']:.3f}")
        print(f"  Std:  {stats['std']:.3f}")
        print(f"  Min:  {stats['min']:.3f}")
        print(f"  Max:  {stats['max']:.3f}")
        print(f"  Valid points: {stats['count_valid']}")
    except (ValueError, KeyError) as e:
        print(f"\n{param.capitalize()}: Not available in dataset")

## 3. Data Conversion and Export

SeaSenseLib can export data to various formats while preserving metadata and following CF conventions:

In [ ]:
# Convert CNV to NetCDF and CSV using the new API
# First, clean up complex attributes that can't be serialized to NetCDF
import copy
clean_profile_data = copy.deepcopy(profile_data)

# Remove or convert complex attributes
for attr_name in list(clean_profile_data.attrs.keys()):
    attr_value = clean_profile_data.attrs[attr_name]
    if isinstance(attr_value, dict):
        # Convert dict to string representation
        clean_profile_data.attrs[attr_name] = str(attr_value)
    elif not isinstance(attr_value, (str, int, float, list, tuple, bytes)):
        # Remove any other complex types
        del clean_profile_data.attrs[attr_name]

# Also clean variable attributes
for var_name in clean_profile_data.data_vars:
    for attr_name in list(clean_profile_data[var_name].attrs.keys()):
        attr_value = clean_profile_data[var_name].attrs[attr_name]
        if isinstance(attr_value, dict):
            clean_profile_data[var_name].attrs[attr_name] = str(attr_value)
        elif not isinstance(attr_value, (str, int, float, list, tuple, bytes)):
            del clean_profile_data[var_name].attrs[attr_name]

# Export using the new API
ssl.write(clean_profile_data, 'demo_output_profile.nc')
print("✓ Exported to NetCDF: demo_output_profile.nc")

ssl.write(profile_data, 'demo_output_profile.csv')
print("✓ Exported to CSV: demo_output_profile.csv")

### 3.1 Verify NetCDF Export

Let's read back the NetCDF file to verify the export worked correctly:

In [ ]:
# Read back the exported NetCDF file using the new API
netcdf_data = ssl.read('demo_output_profile.nc')

print("Exported NetCDF file structure:")
print(f"Dimensions: {dict(netcdf_data.dims)}")
print(f"Variables: {list(netcdf_data.data_vars)}")
print(f"Data preserved: {len(netcdf_data.data_vars)} variables exported successfully")
print("\n✓ NetCDF export/import successful!")

## 4. Data Visualization

SeaSenseLib provides specialized plotting tools for oceanographic data visualization:

### 4.1 Temperature-Salinity (T-S) Diagram

T-S diagrams show the relationship between temperature and salinity with density isolines:

In [ ]:
# Create T-S diagram using the new API
if 'temperature' in profile_data.data_vars and 'salinity' in profile_data.data_vars:
    ssl.plot('ts-diagram', profile_data, title="CTD Profile T-S Diagram")
    print("✓ T-S diagram created")
else:
    print("Temperature and/or salinity not available for T-S diagram")

### 4.2 Vertical Profile Plot

Display the CTD cast as a vertical profile showing how parameters change with depth:

In [ ]:
# Create vertical profile plot using the new API
ssl.plot('depth-profile', profile_data, title="CTD Profile")
print("✓ Vertical profile plot created")

### 4.3 Time Series Plots

For the moored instrument data, let's create time series plots:

In [ ]:
# Single parameter time series - using direct plotter since API has a bug
if 'temperature' in timeseries_data.data_vars:
    ssl.plot('time-series', profile_data, parameters=['temperature'], title="CTD Profile T-S Diagram")
    print("✓ Temperature time series created")
else:
    print("Temperature data not available for time series")

In [ ]:
# Multi-parameter time series - using direct plotter
available_ts_params = [p for p in ['temperature', 'salinity', 'pressure'] if p in timeseries_data.data_vars]

if len(available_ts_params) >= 2:
    ssl.plot('time-series', profile_data, parameters=available_ts_params, title="CTD Profile T-S Diagram")
    print(f"✓ Multi-parameter time series created for: {available_ts_params[:2]}")
else:
    print(f"Insufficient parameters for multi-plot. Available: {available_ts_params}")

## 5. Further data tools

SeaSenseLib provides basic tools for subsetting and statistics:

### 5.1 Data Subsetting

Extract specific depth ranges or time periods:

In [ ]:
# Subset profile data by depth (if pressure/depth data available)
subset_processor = SubsetProcessor(profile_data)

if 'pressure' in profile_data.data_vars:
    # Subset to upper 50 dbar (approximately 50 meters)
    shallow_data = subset_processor.set_parameter_name('pressure').set_parameter_value_min(0).set_parameter_value_max(50).get_subset()
    print(f"Original data points: {len(profile_data.pressure)}")
    print(f"Shallow subset (0-50 dbar): {len(shallow_data.pressure)}")
    print(f"Pressure range in subset: {shallow_data.pressure.min().values:.1f} - {shallow_data.pressure.max().values:.1f} dbar")
else:
    print("Pressure data not available for depth subsetting")

In [ ]:
import pandas as pd
# Subset time series data by time period
if 'time' in timeseries_data.coords and len(timeseries_data.time) > 100:
    ts_subset_processor = SubsetProcessor(timeseries_data)
    # Get the first half of the time series
    start_time = timeseries_data.time.min()
    mid_time = timeseries_data.time[len(timeseries_data.time)//2]
    
    first_half = ts_subset_processor.set_time_min(pd.Timestamp(start_time.values)).set_time_max(pd.Timestamp(mid_time.values)).get_subset()

    print(f"Original time series length: {len(timeseries_data.time)}")
    print(f"First half subset length: {len(first_half.time)}")
    print(f"Time range in subset: {first_half.time.min().values} to {first_half.time.max().values}")
else:
    print("Time data not suitable for subsetting")

### 5.2 Statistical Analysis

Calculate statistics for the datasets:

In [ ]:
# Compare statistics between full profile and shallow subset
if 'pressure' in profile_data.data_vars and 'temperature' in profile_data.data_vars:
    full_temp_processor = StatisticsProcessor(profile_data, 'temperature')
    full_stats = full_temp_processor.get_all_statistics()
    
    shallow_temp_processor = StatisticsProcessor(shallow_data, 'temperature')
    shallow_stats = shallow_temp_processor.get_all_statistics()
    
    print("=== Temperature Statistics Comparison ===")
    print(f"Full profile - Mean: {full_stats['mean']:.3f}°C, Range: {full_stats['min']:.3f} - {full_stats['max']:.3f}°C")
    print(f"Shallow (0-50m) - Mean: {shallow_stats['mean']:.3f}°C, Range: {shallow_stats['min']:.3f} - {shallow_stats['max']:.3f}°C")
    
    temp_diff = shallow_stats['mean'] - full_stats['mean']
    print(f"\nTemperature difference (shallow - full): {temp_diff:.3f}°C")
    if temp_diff > 0:
        print("→ Shallow waters are warmer (typical thermocline pattern)")
    else:
        print("→ Shallow waters are cooler")

## Next Steps

To learn more about SeaSenseLib:

1. **Documentation**: Check the full documentation for detailed API reference
2. **CLI Usage**: Try the command-line interface with `seasenselib --help`
3. **More Formats**: Explore support for RBR, ADCP, and other instrument formats (see the Moored Instruments demo)
4. **Custom Processing**: Implement custom readers and processors for your specific needs
5. **Integration**: Integrate SeaSenseLib into your oceanographic data processing workflows

### Example CLI Commands

```bash
# List supported formats
seasenselib formats

# Convert CNV to NetCDF
seasenselib convert -i examples/sea-practical-2023.cnv -o output.nc

# Create plots
seasenselib plot ts-diagram -i output.nc -o ts_diagram.png
seasenselib plot depth-profile -i output.nc -o profile.png
seasenselib plot time-series -i examples/denmark-strait-ds-m1-17.cnv -p temperature
```

Happy oceanographic data processing! 🌊